In [6]:
from pathlib import Path
path = Path('data/time_stamps')

line_range = (2000, 2100)
pixel_range = (5000, 5100)

In [8]:
# Get shape of the SLCs
shape = []
for file in ['lines', 'pixels']:
    with open(path / 'n{}_crp.txt'.format(file)) as f:
        content = f.readlines()
        shape.append(int(content[0]))
shape = tuple(shape)
print(shape)

(9914, 41174)


In [9]:
dir_slcs = [d / 'slc_srd_nl_veenweiden.raw' for d in path.iterdir() if d.is_dir()]

print(dir_slcs)


[PosixPath('data/time_stamps/20141116/slc_srd_nl_veenweiden.raw'), PosixPath('data/time_stamps/20141128/slc_srd_nl_veenweiden.raw'), PosixPath('data/time_stamps/20141210/slc_srd_nl_veenweiden.raw')]


In [10]:
n_time = len(dir_slcs)
n_points = (line_range[1]-line_range[0]) * (pixel_range[1]-pixel_range[0])

print(n_time)
print(n_points)

3
10000


In [11]:
import numpy as np
import xarray as xr

stack=np.empty((n_points, 0))
for d in dir_slcs:
    slc = np.memmap(d, mode='r', dtype='float32',shape=shape)
    stack = np.append(stack, slc[line_range[0]:line_range[1], pixel_range[0]:pixel_range[1]].reshape(n_points, 1), axis=1)

print(stack)
print(stack.shape)

[[ 35.68050003 -43.69489288 -37.2166748 ]
 [-70.70513916  62.954216   101.8187561 ]
 [  6.88615561  45.29898453  -0.14676106]
 ...
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]
 [  0.           0.           0.        ]]
(10000, 3)


In [12]:
data_xr = xr.Dataset(data_vars=dict(amp=(["points", "time"], stack),),
                     coords=dict(
                        points=np.arange(n_points),
                        time=np.arange(n_time),
                        ))
data_xr

<xarray.Dataset>
Dimensions:  (points: 10000, time: 3)
Coordinates:
  * points   (points) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
  * time     (time) int64 0 1 2
Data variables:
    amp      (points, time) float64 35.68 -43.69 -37.22 -70.71 ... 0.0 0.0 0.0

In [13]:
import stmat
data_xr = data_xr.stm.add_metadata({"name":"test_stm"})

In [14]:
data_xr

<xarray.Dataset>
Dimensions:  (points: 10000, time: 3)
Coordinates:
  * points   (points) int64 0 1 2 3 4 5 6 ... 9993 9994 9995 9996 9997 9998 9999
  * time     (time) int64 0 1 2
Data variables:
    amp      (points, time) float64 35.68 -43.69 -37.22 -70.71 ... 0.0 0.0 0.0
Attributes:
    name:     test_stm

In [15]:
data_xr.stm

In [16]:
data_xr['amp_mean'] = data_xr.mean(dim='time')['amp']

In [17]:
data_xr.stm.subset(method='threshold', var='amp_mean', threshold='<0.4')

<xarray.Dataset>
Dimensions:   (points: 7456, time: 3)
Coordinates:
  * points    (points) int64 0 4 5 10 12 13 14 ... 9994 9995 9996 9997 9998 9999
  * time      (time) int64 0 1 2
Data variables:
    amp       (points, time) float64 35.68 -43.69 -37.22 3.831 ... 0.0 0.0 0.0
    amp_mean  (points) float64 -15.08 -15.72 -23.41 -56.77 ... 0.0 0.0 0.0 0.0
Attributes:
    name:     test_stm